# Microsim Analysis

Does some analysis of the outputs from the [microsim_model.py](./microsim_model.py).

## Initialisation

In [3]:
# Import libraries
import pandas as pd
from typing import List
import matplotlib.pyplot as plt

## Read data

Each time the model is run it outputs data into a new sub-directory under [../data/output/](../data/output) numbered incrementally. E.g.:
 - `data/output/0`
 - `data/output/1
 - ... 
 
The following variables specify which directories to read the data from

In [4]:
START_DIRECTORY = 0
END_DIRECTORY = 9

Now read the data:

In [5]:
XXXX

NameError: name 'XXXX' is not defined

## Disease status

A graph of disease statuses over time ...

In [7]:
XXXX

NameError: name 'XXXX' is not defined

## Locations

Some analysis/visualisation of the locations ... 

In [8]:
XXXX

NameError: name 'XXXX' is not defined